In [7]:
import numpy as np
import cv2, os, pyautogui, time
from PIL import Image
import matplotlib.pyplot as plt
import warnings

from src.face_features import *
from src.tools import *

width, height= pyautogui.size()
img_size = 64
n_channel = 1

# Fast (need large memory)

In [2]:
X_temp, Y_temp = [], []

In [3]:
cap = cv2.VideoCapture(0)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
class GetOutOfLoop(Exception):
    pass


x, y = 0.5, 0.5
image = np.zeros((height, width, 3), np.uint8)
path = [[0,0], [1,0], [0,1], [1,1]]
for i in range(50):
    x_new, y_new = np.random.uniform(), np.random.uniform()
    while abs(x_new - path[-1][0]) + abs(y_new-path[-1][0]) < 1.0:
        x_new, y_new = np.random.uniform(), np.random.uniform()
    path.append([x_new, y_new])
np.random.shuffle(np.array(path))
    
try:
    for x_new, y_new in path:
        T = 1000
        n = 150
        for xc, yc in np.linspace([x, y], [x_new, y_new], n):
            image[:,:,:] = 215
            cv2.circle(image, (int(xc*width),int(yc*height)), radius=15, color=(35,35,35), thickness=-2)
            cv2.imshow("window", image)
            cv2.waitKey(T//n)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                X_temp.append(frame)
                Y_temp.append([xc,yc])
            if cv2.waitKey(1) & 0xFF == ord('q'):
                raise GetOutOfLoop
        x, y = x_new, y_new
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [8]:
from ipywidgets import IntProgress
from IPython.display import display

max_count = len(X_temp)

f = IntProgress(min=0, max=max_count) # instantiate the bar
display(f) # display the bar

X_left, X_right, X_face, X_mask, Y = [], [], [], [], []
for i, frame in enumerate(X_temp):
    face_features = extract_image_features(frame)
    try:
        face_features[1][0]; face_features[1][1]; face_features[0]; face_features[2]
        X_left.append(cv2.resize(face_features[1][0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_right.append(cv2.resize(face_features[1][1], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_face.append(cv2.resize(face_features[0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
        X_mask.append(face_features[2])
        Y.append(Y_temp[i])
    except:
        pass
    f.value += 1

IntProgress(value=0, max=4690)

In [9]:
X_left = np.expand_dims(np.array(X_left), -1)
X_right = np.expand_dims(np.array(X_right), -1)
X_face = np.expand_dims(np.array(X_face), -1)
X_mask = np.array(X_mask)
Y = np.array(Y)
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape)

(3752, 64, 64, 1) (3752, 64, 64, 1) (3752, 64, 64, 1) (3752, 625) (3752, 2)


# Slow (real-time preprocessing)

In [ ]:
X_left, X_right, X_face, X_mask, Y = [], [], [], [], []

In [ ]:
cap = cv2.VideoCapture(0)
# frame_width, frame_height = 320, 240
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
cv2.namedWindow("window", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("window",cv2.WND_PROP_FULLSCREEN,cv2.WINDOW_FULLSCREEN)
class GetOutOfLoop(Exception):
    pass


x, y = 0.5, 0.5
image = np.zeros((height, width, 3), np.uint8)
path = [[0,0], [1,0], [0,1], [1,1]]
for i in range(50):
    x_new, y_new = np.random.uniform(), np.random.uniform()
    while abs(x_new - path[-1][0]) + abs(y_new-path[-1][0]) < 1.0:
        x_new, y_new = np.random.uniform(), np.random.uniform()
    path.append([x_new, y_new])
np.random.shuffle(np.array(path))
    
try:
    for x_new, y_new in path:
        T = 2000
        n = 150
        for xc, yc in np.linspace([x, y], [x_new, y_new], n):
            image[:,:,:] = 215
            cv2.circle(image, (int(xc*width),int(yc*height)), radius=15, color=(35,35,35), thickness=-2)
            cv2.imshow("window", image)
            cv2.waitKey(T//n)
            ret, frame = cap.read()
            if ret:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                face_features = extract_image_features(frame)
                try:
                    face_features[1][0]; face_features[1][1]; face_features[0]; face_features[2]
                    X_left.append(cv2.resize(face_features[1][0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_right.append(cv2.resize(face_features[1][1], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_face.append(cv2.resize(face_features[0], (img_size,img_size), interpolation = cv2.INTER_AREA)/255.0)
                    X_mask.append(face_features[2])
                    Y.append([xc,yc])
                except Exception as e:
                    print(e)
#                 img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
#                 X_temp.append(np.expand_dims(img, -1))
                
#                 Y.append([xc,yc])
            if cv2.waitKey(1) & 0xFF == ord('q'):
                raise GetOutOfLoop
        x, y = x_new, y_new
except Exception as e:
    print(e)
finally:
    cv2.destroyAllWindows()
    cap.release()

In [ ]:
X_left = np.expand_dims(np.array(X_left), -1)
X_right = np.expand_dims(np.array(X_right), -1)
X_face = np.expand_dims(np.array(X_face), -1)
X_mask = np.array(X_mask)
Y = np.array(Y)
print(X_left.shape, X_right.shape, X_face.shape, X_mask.shape, Y.shape)

In [10]:
import pickle

pickle.dump([X_left, X_right, X_face, X_mask, Y], open("save2.pkl", "wb"))